In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/regression-clean-data/Regression-Clean-Data.csv


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

data = pd.read_csv("/kaggle/input/regression-clean-data/Regression-Clean-Data.csv")

In [13]:
data

,Unnamed: 0,Observation,Dist_Taxi,Dist_Market,Dist_Hospital,Carpet,Builtup,Parking,City_Category,Rainfall,House_Price
0,1,1,9796,5250,10703,1659,1961,Open,CAT B,530,6649000
1,2,2,8294,8186,12694,1461,1752,Not Provided,CAT B,210,3982000
2,3,3,11001,14399,16991,1340,1609,Not Provided,CAT A,720,5401000
3,4,4,8301,11188,12289,1451,1748,Covered,CAT B,620,5373000
4,5,5,10510,12629,13921,1770,2111,Not Provided,CAT B,450,4662000
...,...,...,...,...,...,...,...,...,...,...,...
892,928,928,12176,8518,15673,1582,1910,Covered,CAT C,1080,6639000
893,929,929,7214,8717,10553,1387,1663,Open,CAT A,850,8208000
894,930,930,7423,11708,13220,1200,1436,Open,CAT A,1060,7644000
895,931,931,15082,14700,19617,1299,1560,Open,CAT B,770,9661000


In [18]:
data = data.drop(['Unnamed: 0', 'Observation'],axis=1)
data.head()

,Dist_Taxi,Dist_Market,Dist_Hospital,Carpet,Builtup,Parking,City_Category,Rainfall,House_Price
0,9796,5250,10703,1659,1961,Open,CAT B,530,6649000
1,8294,8186,12694,1461,1752,Not Provided,CAT B,210,3982000
2,11001,14399,16991,1340,1609,Not Provided,CAT A,720,5401000
3,8301,11188,12289,1451,1748,Covered,CAT B,620,5373000
4,10510,12629,13921,1770,2111,Not Provided,CAT B,450,4662000


In [8]:
data['Parking'].unique()

array(['Open', 'Not Provided', 'Covered', 'No Parking'], dtype=object)

In [14]:
data['City_Category'].unique()

array(['CAT B', 'CAT A', 'CAT C'], dtype=object)

In [28]:
data = pd.get_dummies(data, columns=['Parking', 'City_Category']).astype(int)
data.head()

,Dist_Taxi,Dist_Market,Dist_Hospital,Carpet,Builtup,Rainfall,House_Price,Parking_Covered,Parking_No Parking,Parking_Not Provided,Parking_Open,City_Category_CAT A,City_Category_CAT B,City_Category_CAT C
0,9796,5250,10703,1659,1961,530,6649000,0,0,0,1,0,1,0
1,8294,8186,12694,1461,1752,210,3982000,0,0,1,0,0,1,0
2,11001,14399,16991,1340,1609,720,5401000,0,0,1,0,1,0,0
3,8301,11188,12289,1451,1748,620,5373000,1,0,0,0,0,1,0
4,10510,12629,13921,1770,2111,450,4662000,0,0,1,0,0,1,0


In [15]:
data.dtypes

Unnamed: 0        int64
Observation       int64
Dist_Taxi         int64
Dist_Market       int64
Dist_Hospital     int64
Carpet            int64
Builtup           int64
Parking          object
City_Category    object
Rainfall          int64
House_Price       int64
dtype: object

## Train data and Test Data

In [31]:
X = data.drop('House_Price', axis=1)
y = data['House_Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Build regression model

In [32]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [79]:
X.corr()

,Dist_Taxi,Dist_Market,Dist_Hospital,Carpet,Builtup,Rainfall,Parking_Covered,Parking_No Parking,Parking_Not Provided,Parking_Open,City_Category_CAT A,City_Category_CAT B,City_Category_CAT C
Dist_Taxi,1.000000,0.453479,0.795520,0.008703,0.008230,0.013540,-0.073926,0.075454,0.042887,-0.033554,0.023590,-0.048558,0.028215
Dist_Market,0.453479,1.000000,0.621466,-0.020778,-0.020384,0.069806,-0.032188,0.040266,0.025954,-0.026574,0.090214,-0.100869,0.013686
Dist_Hospital,0.795520,0.621466,1.000000,0.011706,0.011960,0.046826,-0.070654,0.094325,0.032644,-0.041233,0.056660,-0.071577,0.017731
Carpet,0.008703,-0.020778,0.011706,1.000000,0.998885,-0.043485,0.061586,0.010111,-0.043958,-0.019131,-0.050772,0.047001,0.003146
Builtup,0.008230,-0.020384,0.011960,0.998885,1.000000,-0.043424,0.062265,0.009990,-0.046727,-0.017148,-0.049740,0.046457,0.002624
Rainfall,0.013540,0.069806,0.046826,-0.043485,-0.043424,1.000000,0.015532,-0.054677,-0.029688,0.054268,0.033890,-0.015125,-0.020146
Parking_Covered,-0.073926,-0.032188,-0.070654,0.061586,0.062265,0.015532,1.000000,-0.216384,-0.288203,-0.403612,0.048844,-0.060767,0.014246
Parking_No Parking,0.075454,0.040266,0.094325,0.010111,0.009990,-0.054677,-0.216384,1.000000,-0.248411,-0.347886,0.001093,-0.022295,0.023571
Parking_Not Provided,0.042887,0.025954,0.032644,-0.043958,-0.046727,-0.029688,-0.288203,-0.248411,1.000000,-0.463351,0.006431,0.009181,-0.017208
Parking_Open,-0.033554,-0.026574,-0.041233,-0.019131,-0.017148,0.054268,-0.403612,-0.347886,-0.463351,1.000000,-0.046546,0.058305,-0.014017


## Model LinearRegression

In [42]:
coefficients = model.coef_ # slope
print("Coefficients:", coefficients)

interception = model.intercept_
print("interception:", interception)

y_pred = model.predict(X_test) 
mse = mean_squared_error(y_test, y_pred) # độ sai lệch trung bình bình phương giữa y_test and y_pred
r2 = model.score(X_test, y_test) # đo lường mức độ mà mô hình dự đoán khớp với y_pred and y_test
print("Mean squared error:", mse)
print("R-squared:", r2)

Coefficients: [ 5.62400917e+01  2.87977819e+01  1.41832447e+01  7.41060692e+02
 -5.17212194e+01 -2.35695440e+02  3.33619341e+05 -2.29901481e+05
 -2.13384909e+05  1.09667048e+05  1.56791413e+06 -2.20674525e+05
 -1.34723960e+06]
interception: 3964396.483374444
Mean squared error: 1632480061129.533
R-squared: 0.47729857553419874


## Build 2nd regression model with principal components.

In [53]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Standardize the features (Chuẩn hóa dữ liệu)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [54]:
model_pca = LinearRegression()
model_pca.fit(X_train_pca, y_train_pca)

LinearRegression()

In [55]:
coefficients = model_pca.coef_ # slope
print("Coefficients:", coefficients)

interception = model_pca.intercept_
print("interception:", interception)

y_pred_pca = model_pca.predict(X_test_pca) 
mse = mean_squared_error(y_test, y_pred_pca) # độ sai lệch trung bình bình phương giữa y_test and y_pred
r2 = model_pca.score(X_test_pca, y_test_pca) # đo lường mức độ mà mô hình dự đoán khớp với y_pred and y_test
print("Mean squared error:", mse)
print("R-squared:", r2)

Coefficients: [ 1.50996460e+05  6.33228207e+04  4.16460273e+04  5.15286556e+04
  1.25515771e+05 -7.55693413e+04 -7.33480035e+19 -6.66596334e+19
 -7.91543608e+19 -8.94703774e+19 -9.26824160e+18 -9.44277845e+18
 -8.50189464e+18]
interception: 5867476.255183413
Mean squared error: 1641509696466.153
R-squared: 0.47440738968437435


## Build 3rd regression model with dominant variables in significant principal components: which are variables chosen

In [58]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# calculate the ratio by principal components (Xem tỷ lệ biến thiên giải thích bởi các thành phần chính)
explained_variance = pca.explained_variance_ratio_
cumulative_variance = explained_variance.cumsum()

In [59]:
# Ý Nghĩa: Đây là tỷ lệ phần trăm của tổng biến thiên trong dữ liệu mà mỗi thành phần chính (principal component) giải thích.
# Giải Thích: Nếu explained_variance[i] là 0.2, điều này có nghĩa là thành phần chính thứ i giải thích 20% tổng biến thiên trong dữ liệu.
# Tỷ Lệ Biến Thiên Giải Thích (explained_variance): Giúp xác định độ quan trọng của từng thành phần chính.
explained_variance

array([1.79294696e-01, 1.56350696e-01, 1.24484443e-01, 1.13022612e-01,
       1.09043045e-01, 9.53679129e-02, 9.18585162e-02, 7.41818606e-02,
       4.29469680e-02, 1.33639331e-02, 8.53185014e-05, 1.53597917e-32,
       1.04984560e-32])

In [60]:
# Ý Nghĩa: Đây là tổng tỷ lệ biến thiên giải thích được tích lũy từ các thành phần chính theo thứ tự giảm dần.
# Giải Thích: Nếu cumulative_variance[i] là 0.85, điều này có nghĩa là tổng biến thiên giải thích của các thành phần chính từ đầu đến
# thành phần thứ i là 85%.
# Biến Thiên Tích Lũy (cumulative_variance): Giúp xác định số lượng thành phần chính cần thiết để giải thích một phần lớn biến thiên 
#     trong dữ liệu. Ví dụ, nếu bạn muốn giải thích 95% biến thiên, bạn có thể chọn số lượng thành phần chính cần thiết dựa trên giá trị tích lũy
cumulative_variance

array([0.1792947 , 0.33564539, 0.46012983, 0.57315245, 0.68219549,
       0.7775634 , 0.86942192, 0.94360378, 0.98655075, 0.99991468,
       1.        , 1.        , 1.        ])

In [70]:
import numpy as np

# Xác định số lượng thành phần chính cần thiết (ví dụ: giải thích 95% biến thiên)
n_components = np.argmax(cumulative_variance >= 0.95) + 1

# Chọn các thành phần chính --> cols có cumulative_variance >= 0.95 sẽ bỏ 
X_pca_selected = X_pca[:, :n_components]
X_train_pca_cv, X_test_pca_cv, y_train_pca_cv, y_test_pca_cv = train_test_split(X_pca_selected, y, test_size=0.2, random_state=42)

In [83]:
X_test_pca_cv

array([[ 1.57351525,  2.38314317, -1.97810083, ..., -0.16265392,
        -0.0081332 , -0.65687624],
       [-0.01283397, -0.84203574, -2.01805463, ..., -0.02600299,
        -0.96264956, -0.04207294],
       [ 0.32543957, -0.1355865 ,  0.69311392, ...,  1.67538984,
         1.48729408,  0.89344764],
       ...,
       [-1.01293927,  1.53462358,  1.1473775 , ..., -0.17996479,
         1.44570637, -0.759323  ],
       [ 1.72475597,  2.01330545,  2.38478219, ..., -1.33304524,
         0.79974858,  0.74715358],
       [-0.40360544, -0.97817473, -0.54861309, ...,  0.36938213,
        -1.71466803,  0.09181851]])

In [72]:
model_pca_cv = LinearRegression()
model_pca_cv.fit(X_train_pca_cv, y_train_pca_cv)

LinearRegression()

In [73]:
coefficients = model_pca_cv.coef_ # slope
print("Coefficients:", coefficients)

interception = model_pca_cv.intercept_
print("interception:", interception)

y_pred_pca_cv = model_pca.predict(X_test_pca_cv) 
mse = mean_squared_error(y_test_pca_cv, y_pred_pca_cv) # độ sai lệch trung bình bình phương giữa y_test and y_pred
r2 = model_pca.score(X_test_pca_cv, y_test_pca_cv) # đo lường mức độ mà mô hình dự đoán khớp với y_pred and y_test
print("Mean squared error:", mse)
print("R-squared:", r2)

Coefficients: [-178234.24303134  -59109.51103118  424226.67403518 -364771.28181882
 -785662.70135224   50682.21373902   28334.56751215 -172279.17367963
  -61550.45461559]
interception: 5903150.478852776
Mean squared error: 1689074627476.102
R-squared: 0.47225705126850315
